# Libraries

In [ ]:
import numpy as np
import keras
import csv
from keras.utils import to_categorical
from dlxsudoku import Sudoku

# Functions

### Function that writes to csv file

In [ ]:
def write_csv(header, content, file):    
    with open(file, "w") as write_file:
        writer = csv.writer(write_file)
        writer.writerow(header)
        for row in content:
            writer.writerow(row)

### Function that randomly deletes some digits from the solution

In [ ]:
def delete_digits(X, digits_to_delete=1):
    grids = X.argmax(3)
    for grid in grids:
        grid.flat[np.random.choice(81, digits_to_delete, replace=False)] = 0
    return to_one_hot(grids)

### Function that sums up the difference between true solutions and predicted ones

In [ ]:
def total_diff(true, predicted):
    return np.sum(true != predicted, axis=(1, 2))

### Function that convert from grid to one-hot

In [ ]:
def to_one_hot(x, n_classes=10): 
    return to_categorical(x, num_classes=n_classes).astype('float32')

### Function that convert from one-hot matrix to grid

In [1]:
def from_one_hot(one_hot_solutions):
    return np.argmax(one_hot_solutions, axis=3)

### Function that convert probability vector to one-hot array

In [ ]:
def proba_to_one_hot(proba):
    return np.array(proba == max(proba), dtype=np.float32)

### Function that preprocesses the data

In [ ]:
def preprocess(X, y, from_string=True, reshape=True, one_hot=True):
    # Initial shape
    print("Initial shape of X:", X.shape)
    print("Initial shape of y:", y.shape)
    
    # Make copies
    X_prep = X.copy()
    y_prep = y.copy()
    
    # Convert from string
    def to_int(x):
        return np.array([[int(digit) for digit in row] for row in x])
    if from_string:
        X_prep = to_int(X_prep)
        y_prep = to_int(y_prep)
        print("Convert strings in X to:", X_prep.shape)
        print("Convert strings in y to:", y_prep.shape)
    
    # Reshape to 9x9
    def reshape_f(x):
        return np.reshape(x, (9, 9))
    if reshape:
        X_prep = np.array(list(map(reshape_f, X_prep)))
        y_prep = np.array(list(map(reshape_f, y_prep)))
        print("Reshape X to:", X_prep.shape)
        print("Reshape y to:", y_prep.shape)
        
    # One-hot encoding
    if one_hot:
        X_prep = to_one_hot(X_prep)
        y_prep = to_one_hot(y_prep - 1, n_classes=9)
        print("Shape of one-hot X:", X_prep.shape)
        print("Shape of one-hot y:", y_prep.shape)
        
    return X_prep, y_prep

### Function that generates prediction

In [1]:
def iterative_predict(models, puzzles):
    """
    Our attempt to re-implement this function but failed to optimize
    """
    # solutions are built from puzzles
    solutions = to_one_hot(puzzles.copy())
    
    while True:
        # get zero positions
        zeros = from_one_hot(solutions) == 0
        if np.sum(zeros) == 0:
            break # break if no zero left
        zero_pos = []
        for zero_grid in zeros:
            pos = np.nonzero(zero_grid)
            pos = list(zip(pos[0], pos[1]))
            zero_pos.append(pos)
        
        # get predictions in shape (81, -1, 9) and reshape into (-1, 9, 9, 9)
        preds = np.array(model.predict(solutions))
        preds = np.array([np.reshape(preds[:, i, :], (9, 9, 9)) 
                          for i in range(preds.shape[1])])

        # get best probability for each cell
        max_probs = np.max(preds, axis=3)
            
        # Loop through each puzzle
        for solution, pred, max_prob, zero_p in zip(solutions, preds, max_probs, zero_pos):
            if len(zero_p) > 0: # if there is any zero in this puzzle
                # get the position with largest probability
                best_pos = zero_p[0]
                best_prob = max_prob[best_pos]
                for pos in zero_p:
                    if max_prob[pos] > best_prob:
                        best_prob = max_prob[pos]
                        best_pos = pos
                        
                # fill in best_pos with corresponding value
                # append [0] because puzzle has 10 classes
                solution[best_pos] = np.append([0], proba_to_one_hot(pred[best_pos]))
                
    # convert to normal grid
    return from_one_hot(solutions)

In [ ]:
def efficient_iterative_predict(model, puzzles):
    """
    Adapted from https://www.kaggle.com/dithyrambe/neural-nets-as-sudoku-solvers
    Updated to have slightly better performance
    """
    solutions = puzzles.copy()
    
    # Loop until no zero left
    while True:
        # get blank positions
        zeros = (solutions == 0).reshape((solutions.shape[0], 81))  
        if np.sum(zeros) == 0:
            break
        
        # get predictions
        preds = np.array(model.predict(to_one_hot(solutions, n_classes=10)))  
        
        # get highest probability for each 81 digit to predict
        probs = preds.max(2).T  
        
        # get corresponding values
        values = preds.argmax(2).T + 1  

        # Loop through all puzzles
        for solution, prob, value, zero in zip(solutions, probs, values, zeros):
            if any(zero): # if there is any blank
                # get blank positions
                where = np.where(zero)[0]  
                
                # get the position with largest probability
                best_pos = where[prob[zero].argmax()]
                
                # fill in the corresponding value
                solution.flat[best_pos] = value[best_pos]
                
    return solutions

### Function that evaluates the model on test set

In [ ]:
def evaluate_model(model, X, predict_func=iterative_predict):
    # Reshape X
    X = from_one_hot(X)
    
    # get model prediction using iterative prediction
    # +1 because there are only 9 classes for the prediction (0-8)
    predictions = predict_func(model, X)
    
    # Verify prediction for each puzzle using correct_solution
    validations = [correct_solution(puzzle, solution) 
                   for puzzle, solution in zip(X, predictions)]
    
    # Calculate accuracy
    accuracy = np.mean(validations)

    return {'predictions': predictions,
            'validations': validations,
            'accuracy': accuracy}

### Function that evaluates different versions of model on validation set

In [ ]:
def validation_summary(model_prefix, model_versions, X, y, batch_size=256):
    evaluations = []
    X_completed = to_one_hot(from_one_hot(y) + 1) # completed sudokus for loss
    for version in model_versions:
        # load model
        model = keras.models.load_model(model_prefix + '-' + str(version) + '.h5')
        
        # get loss
        loss = model.evaluate(X_completed, 
                              [y[:, i, j, :] for i in range(9) for j in range(9)],
                              batch_size=batch_size)[0]
        
        # get accuracy
        accuracy = evaluate_model(model, X)['accuracy']
        
        # append all
        evaluations.append([version, loss, accuracy])
    return evaluations

### Function that check if a solution is correct

In [ ]:
def correct_solution(puzzle, solution):
    def is_sudoku_list(lst):
        return max(lst) == 9 and min(lst) == 1 and len(set(lst)) == len(lst)
    
    match_puzzle = (81 - np.sum(puzzle == solution)) == np.sum(puzzle == 0)
    if match_puzzle:
        bad_rows = [row for row in solution if not is_sudoku_list(row)]
        bad_cols = [col for col in solution.T if not is_sudoku_list(col)]
        bad_squares = []
        for i in np.arange(9, step=3):
            for j in np.arange(9, step=3):
                square = [solution[r][c] for r in range(i, i + 3) for c in range(j, j + 3)]
                if not is_sudoku_list(square):
                    bad_squares.append(square)
        return not (bad_rows or bad_cols or bad_squares)
    else:
        return False

### Function that creates training puzzle from solution

In [ ]:
def to_puzzles(solutions):
    puzzles = solutions.copy()
    puzzles = to_one_hot(from_one_hot(puzzles) + 1)
    return np.array(puzzles)

### Function that converts one hot solution to output tensor shape

In [ ]:
def to_output_shape(y_one_hot):
    # Output tensor has shape of (81, None, 9)
    return [y_one_hot[:, i, j, :] for i in range(9) for j in range(9)]

### Function that trains model

In [ ]:
def train_model(model, X_train, y_train, X_val, y_val, n_blanks=[0], epochs=[10], 
                batch_size=256, early_stop_patience=2, save_prefix=None):
    # Early stopping
    early_stop = EarlyStopping(patience=early_stop_patience, verbose=1)
    
    # Initialization
    iteration = 1
    
    # Starting training
    for n_epoch, n_blank in zip(epochs, n_blanks):
        print('Iteration {}:'.format(iteration))
        iteration += 1

        # Input and output
        train_puzzles = delete_digits(X_train, n_blank) # randomly delete n_blank cells
        train_solutions = to_output_shape(y_train) # convert to output tensor shape
        val_puzzles = delete_digits(X_val, n_blank)
        val_solutions = to_output_shape(y_val)
        
        # Train the model
        model.fit(train_puzzles, train_solutions,
                  validation_data=(val_puzzles, val_solutions),
                  batch_size=batch_size,
                  epochs=n_epoch,
                  callbacks=[early_stop],
                  verbose=1)

        # Save model if needed
        if save_prefix:
            keras.models.save_model(model, save_prefix + ("-%d.h5" % n_blank))

### Solve sudoku using additional package

In [ ]:
def solve(sudoku_strings):
    solutions = []
    for puzzle in sudoku_strings:
        try:
            s = Sudoku(puzzle[0])
            s.solve()
            sol = s.to_oneliner()
        except:
            sol = None
        solutions.append([sol])
    return np.array(solutions)